<!-- # Compare Runs, Choose a Model, Deploy the model to a REST API -->

    -Run Hyperparameter Sweep on Training Script
    -Compare the Results of Runs on MLFlow UI
    -Choose the best Model and Register it as a Model
    -Deploy the Model on REST API
    -Build a Container Image suitable for Deployment in a Cloud Platform

In [3]:
import numpy as np
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
import mlflow
from mlflow.models import infer_signature

data_df = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", sep=";")
data_df.head()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [ ]:
x_train = data_df.drop(columns='quality',axis=1).values
y_train = data_df['quality'].values.ravel()

y_test = data_df.drop(columns='quality',axis=1).values
x_test = data_df['quality'].values.ravel()

In [ ]:
x_train,x_valid,y_train,y_valid = train_test_split(x_train,y_train,test_size=0.2,random_state=42)

signature = infer_signature(x_train,y_train)

In [ ]:
#ANN Model

import mlflow.tensorflow


def train_model(params, epochs,  train_x, train_y, valid_x, valid_y, test_x, test_y):

    #Model Architecture
    mean = np.mean(train_x,axis=0)
    var = np.var(train_x,axis=0)

    model = keras.Sequential(
        [
            keras.Input([x_train.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1)
        ]
    )

    #Compile Model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"], momentum=params["momentum"]

    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    #Train ANN Model with LR and Momentum params with MLFlow Tracking 
    with mlflow.start_run(nested=True):
        model.fit(x_train,y_train,validation_data=(x_valid,y_valid), epochs = epochs, batch_size=65)
        
        #Evaluate Model
        eval_result = model.evaluate(valid_x,valid_y,batch_size=65)
        eval_rmse = eval_result[1]

        mlflow.set_tracking_uri(uri = "http://127.0.0.1:5000")
        
        #Log Parameters and Results
        mlflow.log_param(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        #Log the Model
        mlflow.tensorflow.log_model(model, "ANN Model", signature=signature)

        return {"loss" : eval_rmse, "status" : STATUS_OK, "model" : model }




In [22]:
def objective(params):
    #MLFlow will track the parameters and results for each run
     result = train_model(
          params,
          epochs=3,
          train_x=x_train,
          train_y=y_train,
          valid_x=x_valid,
          valid_y=y_valid,
          test_x=x_test,
          test_y=y_test
     )
     return result

In [ ]:
space = {
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
    

}

In [ ]:
mlflow.set_experiment("Wine Quality: ANN Model Selection")
with mlflow.start_run():
    #HyperParameter Search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=type.suggest,
        max_evals=4,
        trials=trials
    )

    #Details of Best Run
    best_run = sorted(trials.results,key=lambda x: x["loss"])[0]

    #Log Best Parameters